# Project 6: Analyzing Stock Sentiment from Twits


### Load Packages

In [58]:
import json
import nltk
import os
import random
import re
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""


print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

In [5]:
print(messages[:5])

['$FITB great buy at 26.00...ill wait', '@StockTwits $MSFT', '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', '$AMD reveal yourself!']


## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [6]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer 
def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    url_pattern = r'http://.*?[\s\n]'
    text = re.sub(pattern = url_pattern,repl = ' ', string = text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    ticker_pattern =  r'\$[^\s\n]*'
    text = re.sub(pattern = ticker_pattern,repl = ' ', string = text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    user_pattern = r'@.*?[\s\n]'
    text = re.sub(pattern = user_pattern,repl = ' ', string = text)

    # Replace everything not a letter with a space
    not_a_letter = r'[^a-zA-Z]+'
    text = re.sub(pattern = not_a_letter,repl = ' ', string = text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split(' ')

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = WordNetLemmatizer()
    tokens = [wnl.lemmatize(word) for word in tokens if len(word)>1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [7]:


tokenized = [preprocess(message) for message in messages]

In [8]:
print(tokenized[:10])
print(preprocess('RT @google Our annual look at the year 1 2 4 in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG sjsjsj'))

[['great', 'buy', 'at', 'ill', 'wait'], [], ['staanalystalert', 'for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'is', 'buy', 'http', 'www', 'stocktargetadvisor', 'com', 'toprating'], ['heard', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits'], ['reveal', 'yourself'], ['why', 'the', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position'], ['bear', 'have', 'reason', 'on', 'to', 'pay', 'more', 'attention', 'http', 'dividendbot', 'com', 'ba'], ['ok', 'good', 'we', 're', 'not', 'dropping', 'in', 'price', 'over', 'the', 'weekend', 'lol'], ['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above'], ['drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'in', 'month', 'back', 'to', 'if', 'bo', 'then', 'bingo', 'what', 'is', 'the', 'odds']]
['rt', 'our', 'annual', 'look', 'at', 'the', 'year', 'in', 'google', 'blogging', 'and',

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [9]:
from collections import Counter,defaultdict


"""
Create a vocabulary by using Bag of words
"""

bow = Counter()
for l in tokenized:
    bow.update(l)

In [10]:
print(bow.most_common(10))
print(len(tokenized))

[('the', 407277), ('to', 396178), ('amp', 395675), ('http', 393223), ('com', 341504), ('utm', 305611), ('is', 288993), ('for', 282474), ('on', 248172), ('of', 218789)]
1548010


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [11]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""


# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
n = len(messages)
freqs = {word:f/n for word,f in bow.items()}
print(freqs['participate'])
# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.00007

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 20

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = dict(bow.most_common(high_cutoff))


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
print(len(filtered_words))

8.204081368983404e-05
{'the': 407277, 'to': 396178, 'amp': 395675, 'http': 393223, 'com': 341504, 'utm': 305611, 'is': 288993, 'for': 282474, 'on': 248172, 'of': 218789, 'and': 215029, 'www': 214888, 'in': 213470, 'this': 204615, 'it': 200108, 'at': 140842, 'will': 129498, 'up': 122989, 'estimize': 119930, 'source': 111574}
6528


In [69]:
print(filtered_words[:10]) 

['great', 'buy', 'ill', 'wait', 'staanalystalert', 'jefferies', 'maintains', 'with', 'rating', 'hold']


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [14]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""


# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word:i for i,word in enumerate(filtered_words,1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {i:word for i,word in enumerate(filtered_words,1) }
# tokenized with the words not in `filtered_words` removed.
filtered = [ [word for word in message if word in filtered_words] for message in tokenized]
print(filtered[15:50])

[['bull', 'have', 'reason', 'pay', 'more', 'attention', 'dividendbot', 'pfe'], ['catalyst', 'continuing', 'new', 'uptrend', 'pill', 'pack', 'buy', 'out', 'amazon', 'prime', 'day', 'earnings', 'test', 'break', 'soon'], ['ha', 'moved', 'check', 'out', 'movement', 'peer', 'dividendbot', 'aapl'], ['staanalystalert', 'mkm', 'partner', 'set', 'price', 'target', 'with', 'rating', 'buy', 'setting', 'target', 'price', 'usd', 'our', 'own', 'verdict', 'buy', 'stocktargetadvisor', 'toprating'], ['think', 'too', 'early', 'tell', 'what', 'going', 'happen', 'monday', 'even', 'with', 'current', 'news', 'about', 'output', 'there', 'still', 'so', 'many'], ['ha', 'current', 'ratio', 'so', 'financially', 'healthy', 'ha', 'no', 'problem', 'meeting', 'obligation', 'chartmill', 'analyze', 'php', 'stocktwits', 'medium', 'fa', 'content', 'health', 'campaign', 'social', 'tracking', 'nke', 'fa', 'key', 'fee'], ['bullish', 'stock', 'watch', 'setup', 'seeitmarket', 'bullish', 'stock', 'twitter', 'twtr', 'michael',

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [15]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [16]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.195457183073759

Finally let's convert our tokens into integer ids which we can pass to the network.

In [17]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [44]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        # LSTM layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=False)
        
        # dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(lstm_size, output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        return (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        batch_size = nn_input.size(1)

        # embeddings and lstm_out
        x = nn_input.long()
        embeds = self.embedding(x)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # softmax function
        log_ps = self.log_softmax(out)
        
        # reshape to be batch_size last
        log_ps = log_ps.view(-1, batch_size, self.output_size)
        log_ps = log_ps[-1, :, :] # get last batch of labels
        
        # return last softmax output and hidden state
        return log_ps, hidden_state

### View Model

In [45]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.4115, -2.0546, -1.3978, -1.5238, -1.8136],
        [-1.4146, -2.0322, -1.4002, -1.5291, -1.8160],
        [-1.3957, -2.1470, -1.4137, -1.4670, -1.8222],
        [-1.3909, -2.1041, -1.4199, -1.4249, -1.9179]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [46]:
def dataloader(messages, labels, sequence_length=30, batch_size=512, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [56]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
train_percent = 0.8 
split = int(len(sentiments)*train_percent)
train_features = token_ids[:split]
valid_features = token_ids[split:]
train_labels = sentiments[:split]
valid_labels = sentiments[split:]
print(len(sentiments),split)

1032267 825813


In [48]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(6529, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

In [59]:
epochs = 3
batch_size = 512
learning_rate = 0.001
clip = 5 # gradient clipping
sequence_length = 20

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))

    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, 
            sequence_length=sequence_length, shuffle=True):
        steps += 1
        # initialize hidden state
        real_batch_size = min(batch_size, text_batch.size(1))
        hidden = model.init_hidden(real_batch_size)
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        logps, hidden = model(text_batch, hidden)

        # calculate the loss and perform backprop
        loss = criterion(logps.squeeze().view(real_batch_size,-1), labels)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip:
            nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if steps % print_every == 0:
            model.eval()
            num_correct = 0
            
            # Get validation loss
            val_losses = []
            for text_batch, labels in dataloader(
                    valid_features, valid_labels, batch_size=batch_size, 
                    sequence_length=sequence_length, shuffle=False):

                real_batch_size = min(batch_size, text_batch.size(1))
                val_h = model.init_hidden(real_batch_size)

                text_batch, labels = text_batch.to(device), labels.to(device)
                for each in val_h:
                    each.to(device)

                #print(text_batch.size())
                logps, val_h = model(text_batch, val_h)
                val_loss = criterion(logps, labels)
                val_losses.append(val_loss.item())

                # convert output probabilities to predicted class (0 or 4)
                pred = torch.argmax(torch.exp(logps), dim=-1)

                # compare predictions to true label
                correct_tensor = pred.eq(labels.view_as(pred))
                correct = np.squeeze(correct_tensor.numpy()) if device.type=='cpu' \
                     else np.squeeze(correct_tensor.cpu().numpy())
                num_correct += np.sum(correct)

            val_acc = num_correct / len(valid_labels)
            
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Val Accuracy: {:.3f}".format(val_acc))
            model.train()

Starting epoch 1
Epoch: 1/3... Step: 100... Loss: 0.881532... Val Loss: 0.882121 Val Accuracy: 0.654
Epoch: 1/3... Step: 200... Loss: 0.824891... Val Loss: 0.846380 Val Accuracy: 0.673
Epoch: 1/3... Step: 300... Loss: 0.868842... Val Loss: 0.821312 Val Accuracy: 0.683
Epoch: 1/3... Step: 400... Loss: 0.760341... Val Loss: 0.802704 Val Accuracy: 0.687
Epoch: 1/3... Step: 500... Loss: 0.765535... Val Loss: 0.792197 Val Accuracy: 0.692
Epoch: 1/3... Step: 600... Loss: 0.709128... Val Loss: 0.774350 Val Accuracy: 0.703
Epoch: 1/3... Step: 700... Loss: 0.739519... Val Loss: 0.767249 Val Accuracy: 0.703
Epoch: 1/3... Step: 800... Loss: 0.673855... Val Loss: 0.754683 Val Accuracy: 0.709
Epoch: 1/3... Step: 900... Loss: 0.734646... Val Loss: 0.756878 Val Accuracy: 0.707
Epoch: 1/3... Step: 1000... Loss: 0.666754... Val Loss: 0.747916 Val Accuracy: 0.714
Epoch: 1/3... Step: 1100... Loss: 0.779867... Val Loss: 0.741555 Val Accuracy: 0.716
Epoch: 1/3... Step: 1200... Loss: 0.683663... Val Loss: 0

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [60]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    
    tokens = preprocess(text)
    
    # Filter non-vocab words and convert to ids
    tokens =  [vocab[word] for word in tokens if word in filtered_words]
    
        
    token_tensor = torch.tensor(tokens)
    batch_size = 1
    text_input = torch.zeros((sequence_length, batch_size), dtype=torch.int64)
    # padding
    start_idx = max(sequence_length - len(token_tensor), 0)
    text_input[start_idx:, 0] = token_tensor[:sequence_length]
    # Get the model output
    hidden = model.init_hidden(batch_size)
    for each in hidden:
        each.to("cpu")
    logps, _ = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [61]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[ 0.0000,  0.0040,  0.0049,  0.9466,  0.0445]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

The answer (in range [-2,2]) is 1, the uncertainty is 1-0.9466

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [62]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [63]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [64]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [65]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.1627,  0.0134,  0.0476,  0.0549,  0.7214]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.